In [ ]:
conda install -c conda-forge gdown

In [ ]:
import gdown
url = 'https://drive.google.com/uc?id=1rcz0ZRJ0uLoEucq8mposcsUOqFn202bf'
output = 'file.mat'
gdown.download(url, output, quiet=False)

In [ ]:
from scipy.io import loadmat
annots = loadmat('file.mat')
data = [item[0:] for item in annots['h']]

In [ ]:
import random
import os
import tensorflow as tf
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Conv1D
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Embedding, BatchNormalization, LayerNormalization
import tensorflow_addons as tfa
from keras.layers import Concatenate, LeakyReLU
from keras.layers import Reshape
from keras.utils.vis_utils import plot_model
from numpy.random import randint
import numpy as np
import time
#from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier

In [ ]:
def seed_everything(seed=2020):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
seed_everything(42)

In [ ]:
data_array = np.array(data)
data_env1 = data_array[:, :, 0]
dataset = []
num_angles = 90
angle = 0
for angle in range(num_angles):
    angle_i_data = data_env1[:, angle]
    dataset.append(angle_i_data)
dataset = np.array(dataset)

In [ ]:
def define_discriminator(latent_dim,n_classes = 90):
    # label input
    in_label = Input(shape=(1,))
    #embedding for categorical input
    li = Embedding(n_classes , 25)(in_label)
    #linear multiplication
    n_nodes = 1
    li = Dense(n_nodes)(li)
    li = Reshape((n_nodes, 1))(li)
    in_dim = Input(shape=(latent_dim, 1))
    merge = Concatenate()([in_dim,li])
    merge = Conv1D(filters=256, kernel_size=5, padding='same')(merge)
    merge = BatchNormalization()(merge, training=True)
    merge = tf.nn.relu(merge)
    merge = Dense(64, activation = 'relu',  kernel_initializer='he_uniform')(merge)
    merge = Conv1D(filters=128, kernel_size=3, padding='same')(merge)
    merge = BatchNormalization()(merge, training=True)
    merge = tf.nn.relu(merge)
    merge = Dense(64, activation = 'relu',  kernel_initializer='he_uniform')(merge)
    merge = Conv1D(filters=64, kernel_size=3, padding='same')(merge)
    merge = BatchNormalization()(merge, training=True)
    merge = tf.nn.relu(merge)
    merge = Dense(64, activation = 'relu',  kernel_initializer='he_uniform')(merge)
    merge = Conv1D(filters=64, kernel_size=3, padding='same')(merge)
    merge = BatchNormalization()(merge, training=True)
    merge = tf.nn.relu(merge)
    merge = Dense(64, activation = 'relu',  kernel_initializer='he_uniform')(merge)
    merge = Conv1D(filters=64, kernel_size=3, padding='same')(merge)
    merge = BatchNormalization()(merge, training=True)
    merge = tf.nn.relu(merge)
    merge = Dense(64, activation = 'relu',  kernel_initializer='he_uniform')(merge)
    merge = Conv1D(filters=16, kernel_size=3, padding='same')(merge)
    merge = BatchNormalization()(merge, training=True)
    merge = tf.nn.relu(merge)
    merge = Dense(100, activation='relu')(merge)
    out_layer = Dense(1, activation='sigmoid')(merge)
    model = Model([in_dim, in_label], out_layer)
    loss = tf.keras.losses.BinaryCrossentropy()
    #model.compile(loss='mean_absolute_percentage_error', optimizer=tf.keras.optimizers.Adam(lr=1e-4, beta_1 = 0.901), metrics=['accuracy'])
    model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(lr=0.0002, beta_1 = 0.901), metrics=['accuracy'])
    return model

In [ ]:
def define_generator(latent_dim_gen, n_classes = 90, n_outputs = 1):
    # label input
    in_label = Input(shape=(1,))
    # embedding for categorical input
    li = Embedding(n_classes ,25)(in_label)
    # linear multiplication
    n_nodes = 15
    li = Dense(n_nodes)(li)
    # image generator input
    in_lat = Input(shape=(latent_dim_gen,))
    gen = Dense(n_nodes, activation = 'relu',  kernel_initializer='he_uniform')(in_lat)
    gen = Reshape((1,15))(gen)
    merge = Concatenate()([gen,li])
    merge = Conv1D(filters=256, kernel_size=5, padding='same')(merge)
    merge = BatchNormalization()(merge, training=True)
    merge = tf.nn.relu(merge)
    merge = Dense(32, activation = 'relu',  kernel_initializer='he_uniform')(merge)
    merge = Conv1D(filters=128, kernel_size=3, padding='same')(merge)
    merge = BatchNormalization()(merge, training=True)
    merge = tf.nn.relu(merge)
    merge = Dense(32, activation = 'relu',  kernel_initializer='he_uniform')(merge)
    merge = Conv1D(filters=64, kernel_size=3, padding='same')(merge)
    merge = BatchNormalization()(merge, training=True)
    merge = tf.nn.relu(merge)
    merge = Dense(32, activation = 'relu',  kernel_initializer='he_uniform')(merge)
    merge = Conv1D(filters=16, kernel_size=3, padding='same')(merge)
    merge = BatchNormalization()(merge, training=True)
    merge = tf.nn.relu(merge)
    out_layer = Dense(n_outputs, activation='linear')(merge)
    model = Model([in_lat, in_label], out_layer)
    return model

In [ ]:
def GAN_model(generator, discriminator):
    discriminator.trainable = False
    gen_noise, gen_label = generator.input
    gen_output = generator.output
    gan_output = discriminator([gen_output, gen_label])
    model = Model([gen_noise, gen_label], gan_output)
    #model.compile(loss='mean_absolute_percentage_error', optimizer=tf.keras.optimizers.Adam(lr=1e-4, beta_1 = 0.901), metrics=['accuracy'])
    loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.7015)
    model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(lr=0.0002, beta_1 = 0.901), metrics=['accuracy'])
    return model

In [ ]:
X = []
y = []
angle_data = []
angle_num = []
for i in range(90):
    angle_data = np.append(angle_data, dataset[i])
    angle_num = np.append(angle_num, [i]*100000)
angle_num = angle_num.astype(int)

In [ ]:
#n = len(dataset)
import random
def generate_real_samples(angle_data, angle_num, n_samples):
	# choose random instances
	ix = randint(0, angle_data.shape[0], n_samples)
	# select images and labels
	angle_data, angle_num = angle_data[ix], angle_num[ix]
	# generate class labels
	y = ones((n_samples, 1))
	return [angle_data, angle_num], y

In [ ]:
#Generate points in latent space as input for the generator
def generate_latent_points(latent_dim_gen, n, n_classes = 89):
    # generate points in the latent space
    x_input = randn(latent_dim_gen * n)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n, latent_dim_gen)
    labels = randint(0, n_classes, n)
    return [x_input, labels]

In [ ]:
def generate_fake_samples(generator, latent_dim_gen, n):
        # generate points in latent space
        data, angles = generate_latent_points(latent_dim_gen, n)
        #print(data.shape)
        #print(angles.shape)
        # predict outputs
        X = generator.predict([data, angles])
        # create class labels
        y = zeros((n, 1))
        return [X,angles], y

In [ ]:
def train(g_model, d_model, gan_model, latent_dim, latent_dim_gen, angle_data, angle_num, n_epochs=430, n_batch=256):
        # determine half the size of one batch, for updating the discriminator
        half_batch = int(n_batch / 2)
        # manually enumerate epochs
        for i in range(n_epochs):
                # prepare real samples
                [x_real,angle_real], y_real = generate_real_samples(angle_data, angle_num, half_batch)
                # prepare fake examples
                [x_fake,angle_fake], y_fake = generate_fake_samples(g_model, latent_dim_gen, half_batch)
                # update discriminator
                d_model.train_on_batch([x_real, angle_real], y_real)
                d_model.train_on_batch([x_fake, angle_fake], y_fake)
                # prepare points in latent space as input for the generator
                [x_input, angle_input] = generate_latent_points(latent_dim_gen, n_batch)
                # create inverted labels for the fake samples
                y_gan = ones((n_batch, 1))
                # update the generator via the discriminator's error
                gan_model.train_on_batch([x_input, angle_input], y_gan)
                # evaluate the model every n_eval epochs
        gan_model.save('cgan_generator.h')  

In [ ]:
def real_training(angle_num, angle_data):
    try:
        os.mkdir(f'Gen-{angle_num[0]//9}')
    except:
        pass
    latent_dim = 1
    latent_dim_gen = 1
    # create the discriminator
    discriminator = define_discriminator(latent_dim)
    # create the generator
    generator = define_generator(latent_dim_gen)
    # create the gan
    gan_model = GAN_model(generator, discriminator)
    # train model
    train(generator, discriminator, gan_model, latent_dim, latent_dim_gen, angle_data, angle_num)
    generator.save_weights(f'./Gen-{angle_num[0]}/generator-90')
    print(f"Training angle {angle_num[0]//9} done", end='\r')

In [ ]:
for i in range(0, 90, 9):
    angle_n = angle_num[i*angle_num.shape[0]//90: (i+9)*angle_num.shape[0]//90]
    angle_d = angle_data[i*angle_num.shape[0]//90: (i+9)*angle_num.shape[0]//90]
    real_training(angle_n, angle_d)

In [ ]:
i = 4 #choosing angle 4
angle_i_data = dataset[i]
data_hist = np.array(angle_i_data)
data_hist.shape
plt.hist(data_hist, bins=300)
plt.title('Column 85 - Env3 - Real')
plt.savefig('Real-85-3.png')
plt.show()

In [ ]:
def generate_sample_space(latent_dim_gen, angle, n = 100000, n_classes = 10):
    # generate points in the latent space
    x_input = randn(latent_dim_gen * n)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n, latent_dim_gen)
    labels = [angle] * n
    labels = np.array(labels)
    return [x_input, labels]

In [ ]:
def generate_samples(angle, latent_dim = 1, n=100000):
    latent_pts, labels = generate_sample_space(latent_dim, angle)
    #print(latent_pts.shape)
    #print(labels.shape)
    generator = define_generator(latent_dim)
    generator.load_weights(f'./Gen-{angle//9}/generator-90')
    X = generator.predict([latent_pts, labels])
    y = zeros((n, 1))
    return X, y

In [ ]:
x_fake, _ = generate_samples(4)
x_fake = x_fake.reshape(100000,1)
plt.hist(x_fake, bins=300)
plt.title('Column 85 - Env3 - Generated')
plt.savefig('Gen-85-3.png')
plt.show()

In [ ]:
for angle in range(90):
    angle_i_data = dataset[angle]
    data_hist = np.array(angle_i_data)
    print('='*10+f'angle-{angle+1}'+'='*10)
    print(f'Real_Mean:{np.mean(data_hist)}; Real_Variance:{np.std(data_hist)**2}')
    x_fake, _ = generate_samples(angle)
    print(f'Fake_Mean:{np.mean(x_fake)}; Fake_Variance:{np.std(x_fake)**2}')

In [ ]:
 for angle in range(90):
        x_fake, _ = generate_samples(angle)
        x_fake = x_fake.reshape(100000,1)
        plt.subplot(1,2,1)
        plt.hist(x_fake, bins=300)
        print(f'Angle:{angle}, Generated') 
        angle_i_data = dataset[angle]
        data_hist = np.array(angle_i_data)
        data_hist.shape
        plt.subplot(1,2,2)
        plt.hist(data_hist, bins=300)
        print(f'Angle:{angle}, Real') 
        plt.show()